# Bibliotecas

In [ ]:
# Instalación de biblioteca para trabajar con imágenes satelitales
!pip install rasterio

# Sobre la imágenes satelitales

La imagen que trabajaremos fue obtenida por la misión [Sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) llevada acabo por la ESA. 

El archivo `CUraster.tif` es un recorte de la imagen del día 21-07-2021. La imagen tampoco contiene todas las bandas de información que contiene una imagen de Sentinel-2, solo contiene la información del las bandas 8 (cercano al infrarojo), 4 (roja), 3 (verde) y 2 (azul).

Nuestro objetivo será calcular el índice NDVI (Normalized difference vegetation index_) el cual para las imágenes Sentinel-2 se define como: 

$$
\textrm{NDVI}=\frac{B8-B4}{B8+B4}=\frac{\textrm{NIR}-\textrm{RED}}{\textrm{NIR}+\textrm{RED}}
$$


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import rasterio

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
IMG_PATH = '/gdrive/My Drive/ColabData/CUraster.tiff'